# J-Quantsを使って株価情報取得

### 新規Refresh Tokenの取得

設定ファイルとしてファイル名「jpx_jquants_settings.ini」を作り、
```ini
[JPX]
MailAddress=（J-Quantsで登録したメールアドレス）
Password=（J-Quantsで登録したパスワード）
```
と記載する必要があります

In [2]:
import requests
import json
import configparser


config = configparser.ConfigParser()
config_token = configparser.ConfigParser()

config.read("jpx_jquants_settings.ini")
config_token.read("jpx_jquants_tokens.ini")

# Refresh Tokenの取得
data = {"mailaddress":config["JPX"]["MailAddress"], "password":config["JPX"]["Password"]}
r_post = requests.post("https://api.jquants.com/v1/token/auth_user", json=data)
token_json_data = r_post.json()
refresh_token_jpx_jquants = token_json_data["refreshToken"]
# print(token_json_data)

# トークンの保存
try:
    config_token.add_section("JPX_JQUANTS_TOKEN")

# トークンのセクションが存在する場合は更新
except configparser.DuplicateSectionError:
    config_token.set("JPX_JQUANTS_TOKEN", "RefreshToken", refresh_token_jpx_jquants)
    with open("jpx_jquants_tokens.ini", "w") as f:
        config_token.write(f)

    print("Refresh Token is updated.")


Refresh Token is updated.


### IDトークン取得

In [3]:
import configparser
import requests

# ID Tokenの取得
try: 
    config_token == None

except NameError:
    config_token = configparser.ConfigParser()
    config_token.read("jpx_jquants_tokens.ini")

# IDトークンの取得
get_id_token = requests.post(\
    "https://api.jquants.com/v1/token/auth_refresh?refreshtoken={}".format( \
    config_token["JPX_JQUANTS_TOKEN"]["RefreshToken"]))
id_token_json_data = get_id_token.json()
id_token_jpx_jquants = id_token_json_data["idToken"]
config_token.set("JPX_JQUANTS_TOKEN", "IdToken", id_token_jpx_jquants)

# ID Tokenの保存
with open("jpx_jquants_tokens.ini", "w") as f:
    config_token.write(f)

print("ID Token is updated.")

ID Token is updated.


### 銘柄リストを取得してJsonとして保存

In [4]:
import json

headers = {'Authorization': 'Bearer {}'.format(id_token_jpx_jquants)}
get_stock_list = requests.get("https://api.jquants.com/v1/listed/info", headers=headers)

with open("stock_list.json", "w") as f:
    json.dump(get_stock_list.json()['info'], f, ensure_ascii=False)

### 銘柄リストをCSVに出力

In [5]:
import pandas as pd
df = pd.read_json("stock_list.json")
# df = pd.read_json(get_stock_list.json())
df.to_csv("stock_list.csv",encoding='cp932')
print(df.CompanyName)

0                                                      極洋
1                 大和アセットマネジメント株式会社　ｉＦｒｅｅＥＴＦ　ＴＯＰＩＸ（年１回決算型）
2                野村アセットマネジメント株式会社　ＮＥＸＴ　ＦＵＮＤＳ　ＴＯＰＩＸ連動型上場投信
3                     日興アセットマネジメント株式会社　　上場インデックスファンドＴＯＰＩＸ
4       野村アセットマネジメント株式会社　ＮＥＸＴ　ＦＵＮＤＳ　ＣｈｉｎａＡＭＣ・中国株式・上証５０...
                              ...                        
4399                                                 ジェコス
4400                                                 ヤマザワ
4401                                                  やまや
4402                                                サトー商会
4403                                                 ベルーナ
Name: CompanyName, Length: 4404, dtype: object


### 任天堂の株価取得

In [9]:
import requests
import json
import time

stock_list = {'任天堂':79740}

headers = {'Authorization': 'Bearer {}'.format(id_token_jpx_jquants)}
r = requests.get("https://api.jquants.com/v1/prices/daily_quotes?code={0}&from=20230101&to={1}".format(stock_list['任天堂'], time.strftime("%Y%H%M", time.localtime())), headers=headers)
gotStockData = r.json()

nintendo_stock = pd.DataFrame.from_dict(gotStockData['daily_quotes'])
nintendo_stock.to_csv("nintendo_stock.csv",encoding='cp932')

pd.options.plotting.backend = "plotly"
nintendo_stock.plot(x='Date', y='Close', title='Nintendo Stock Price', labels={'close':'Stock Price'}, template='plotly_dark')